In [162]:
import numpy as np
import pandas as pd
import QuantLib as ql
import scipy.stats as sst

In [163]:
# Analytic price
S = 100; r = 0.03; vol = 0.2; T = 1; K = 100; B = 130; rebate = 0
barrierType = ql.Barrier.UpOut; optionType = ql.Option.Call

#Barrier Option
today = ql.Date().todaysDate(); maturity = today + ql.Period(T, ql.Years)

payoff = ql.PlainVanillaPayoff(optionType, K)
euExercise = ql.EuropeanExercise(maturity)
barrierOption = ql.BarrierOption(barrierType, B, rebate, payoff, euExercise)

#Market
spotHandle = ql.QuoteHandle(ql.SimpleQuote(S))
flatRateTs = ql.YieldTermStructureHandle(ql.FlatForward(today, r, ql.Actual365Fixed()))
flatVolTs = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, ql.NullCalendar(), vol, ql.Actual365Fixed()))
bsm = ql.BlackScholesProcess(spotHandle, flatRateTs, flatVolTs)
analyticBarrierEngine = ql.AnalyticBarrierEngine(bsm)

#Pricing
barrierOption.setPricingEngine(analyticBarrierEngine)
analytic_price = barrierOption.NPV()
analytic_price

3.2027496755342333

In [164]:
# Parameter setting
s, k, r, q, t, sigma, option_flag, b, barrier_flag = S, K, r, 0, T, vol, 'call', B, 'up-out'
n,m = 10000, 1000
dt = t/m

In [165]:
# Sampling
z = np.random.standard_normal( m*n ).reshape(n,m)
lhs = sst.qmc.LatinHypercube(d=m).random(n=n)
z_lhs = sst.norm.ppf(lhs)
z_anti = np.concatenate([z,-z], axis=0)
z_lhs_anti = np.concatenate([z_lhs,-z_lhs], axis=0)

In [166]:
# Underlying path
path = s*np.exp((r-q-0.5*(sigma**2))*dt+sigma*np.sqrt(dt)*z).cumprod(axis=1)
path_lhs = s*np.exp((r-q-0.5*(sigma**2))*dt+sigma*np.sqrt(dt)*z_lhs).cumprod(axis=1)
path_anti = s*np.exp((r-q-0.5*(sigma**2))*dt+sigma*np.sqrt(dt)*z_anti).cumprod(axis=1)
path_lhs_anti = s*np.exp((r-q-0.5*(sigma**2))*dt+sigma*np.sqrt(dt)*z_lhs_anti).cumprod(axis=1)

z_moment = ( z.cumsum(axis=1) - z.mean(axis=1).reshape(len(z),1) ) / z.std(axis=1,ddof=1).reshape(len(z),1)
z_lhs_moment = ( z_lhs.cumsum(axis=1) - z_lhs.mean(axis=1).reshape(len(z_lhs),1) ) / z_lhs.std(axis=1,ddof=1).reshape(len(z_lhs),1)
z_anti_moment = ( z_anti.cumsum(axis=1) - z_anti.mean(axis=1).reshape(len(z_anti),1) ) / z_anti.std(axis=1,ddof=1).reshape(len(z_anti),1)
z_lhs_anti_moment = ( z_lhs_anti.cumsum(axis=1) - z_lhs_anti.mean(axis=1).reshape(len(z_lhs_anti),1) ) / z_lhs_anti.std(axis=1,ddof=1).reshape(len(z_lhs_anti),1)

path_moment = s * np.exp((r-q-0.5*(sigma**2))*dt*(np.arange(m)+1) + sigma*np.sqrt(dt)*z_moment)
path_lhs_moment = s * np.exp((r-q-0.5*(sigma**2))*dt*(np.arange(m)+1) + sigma*np.sqrt(dt)*z_lhs_moment)
path_anti_moment = s * np.exp((r-q-0.5*(sigma**2))*dt*(np.arange(m)+1) + sigma*np.sqrt(dt)*z_anti_moment)
path_lhs_anti_moment = s * np.exp((r-q-0.5*(sigma**2))*dt*(np.arange(m)+1) + sigma*np.sqrt(dt)*z_lhs_anti_moment)

In [167]:
# Up and Out Call
logic = path.max(axis=1)<=b
logic_lhs = path_lhs.max(axis=1)<=b
logic_anti = path_anti.max(axis=1)<=b
logic_moment = path_moment.max(axis=1)<=b
logic_lhs_anti = path_lhs_anti.max(axis=1)<=b
logic_lhs_moment = path_lhs_moment.max(axis=1)<=b
logic_anti_moment = path_anti_moment.max(axis=1)<=b
logic_lhs_anti_moment = path_lhs_anti_moment.max(axis=1)<=b

plain = np.maximum(path[:,-1]-k,0) * np.exp(-r*t)
plain_lhs = np.maximum(path_lhs[:,-1]-k,0) * np.exp(-r*t)
plain_anti = np.maximum(path_anti[:,-1]-k,0) * np.exp(-r*t)
plain_moment = np.maximum(path_moment[:,-1]-k,0) * np.exp(-r*t)
plain_lhs_anti = np.maximum(path_lhs_anti[:,-1]-k,0) * np.exp(-r*t)
plain_lhs_moment = np.maximum(path_lhs_moment[:,-1]-k,0) * np.exp(-r*t)
plain_anti_moment = np.maximum(path_anti_moment[:,-1]-k,0) * np.exp(-r*t)
plain_lhs_anti_moment = np.maximum(path_lhs_anti_moment[:,-1]-k,0) * np.exp(-r*t)

barrier = logic * plain
barrier_lhs = logic_lhs * plain_lhs
barrier_anti = logic_anti * plain_anti
barrier_moment = logic_moment * plain_moment
barrier_lhs_anti = logic_lhs_anti * plain_lhs_anti
barrier_lhs_moment = logic_lhs_moment * plain_lhs_moment
barrier_anti_moment = logic_anti_moment * plain_anti_moment
barrier_lhs_anti_moment = logic_lhs_anti_moment * plain_lhs_anti_moment

In [168]:
# Control variate
d1 = (np.log(s/k) + (r - q + 0.5*sigma**2)*t) / (sigma*np.sqrt(t))
d2 = d1 - sigma*np.sqrt(t)
nd1, nd2 = sst.norm.cdf(d1), sst.norm.cdf(d2)
plain_analytic = s*np.exp(-q*t)*nd1 - k*np.exp(-r*t)*nd2

result = pd.DataFrame(columns=['barrier','bias','var','mse','barrierCV','biasCV','beta','plain'],
                      index=['basic','lhs','anti','moment','lhs_anti','lhs_moment','anti_moment','lhs_anti_lhs'])

In [169]:
result['barrier']=[barrier.mean(),
                   barrier_lhs.mean(),
                   barrier_anti.mean(),
                   barrier_moment.mean(),
                   barrier_lhs_anti.mean(),
                   barrier_lhs_moment.mean(),
                   barrier_anti_moment.mean(),
                   barrier_lhs_anti_moment.mean()]
result['var']=[barrier.var(ddof=1) / n,
               barrier_lhs.var(ddof=1) / n,
               barrier_anti.var(ddof=1) / n,
               barrier_moment.var(ddof=1) / n,
               barrier_lhs_anti.var(ddof=1) / n,
               barrier_lhs_moment.var(ddof=1) / n,
               barrier_anti_moment.var(ddof=1) / n,
               barrier_lhs_anti_moment.var(ddof=1) / n]
result['plain']=[plain.mean(),
                 plain_lhs.mean(),
                 plain_anti.mean(),
                 plain_moment.mean(),
                 plain_lhs_anti.mean(),
                 plain_lhs_moment.mean(),
                 plain_anti_moment.mean(),
                 plain_lhs_anti_moment.mean()]
result['beta']=[np.cov(barrier,plain,ddof=1)[0,1] / plain.var(ddof = 1),
                np.cov(barrier_lhs,plain_lhs,ddof=1)[0,1] / plain_lhs.var(ddof = 1),
                np.cov(barrier_anti,plain_anti,ddof=1)[0,1] / plain_anti.var(ddof = 1),
                np.cov(barrier_moment,plain_moment,ddof=1)[0,1] / plain_moment.var(ddof = 1),
                np.cov(barrier_lhs_anti,plain_lhs_anti,ddof=1)[0,1] / plain_lhs_anti.var(ddof = 1),
                np.cov(barrier_lhs_moment,plain_lhs_moment,ddof=1)[0,1] / plain_lhs_moment.var(ddof = 1),
                np.cov(barrier_anti_moment,plain_anti_moment,ddof=1)[0,1] / plain_anti_moment.var(ddof = 1),
                np.cov(barrier_lhs_anti_moment,plain_lhs_anti_moment,ddof=1)[0,1] / plain_lhs_anti_moment.var(ddof = 1)]
result['bias'] = (result['barrier'] - analytic_price)**2
result['mse'] = result['bias']+result['var']
result['barrierCV'] = result['barrier'] - result['beta'] * (result['plain'] - plain_analytic)
result['biasCV'] = (result['barrierCV'] - analytic_price)**2
result['CVeffect'] = result['biasCV']<result['bias']

In [170]:
result.sort_values(by='mse')

,barrier,bias,var,mse,barrierCV,biasCV,beta,plain,CVeffect
lhs_anti_lhs,3.298947,0.009254,0.003807,0.013061,3.298681,0.009203,0.089849,9.416368,True
lhs_anti,3.301786,0.009808,0.003814,0.013622,3.301350,0.009722,0.090107,9.418251,True
lhs_moment,3.337133,0.018059,0.003891,0.021950,3.335998,0.017755,0.092363,9.425687,True
anti_moment,3.346964,0.020798,0.003859,0.024657,3.348224,0.021163,0.092287,9.399753,False
lhs,3.350660,0.021877,0.003917,0.025794,3.349826,0.021631,0.093653,9.422307,True
anti,3.353930,0.022855,0.003865,0.026720,3.355402,0.023303,0.092620,9.397513,False
moment,3.354683,0.023084,0.003884,0.026968,3.373520,0.029163,0.097346,9.219901,False
basic,3.355871,0.023446,0.003867,0.027314,3.374740,0.029581,0.096575,9.218026,False
